<a href="https://colab.research.google.com/github/ereinha/SineKAN/blob/main/SineKAN_MNIST_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sine_kan import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Load MNIST
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
trainset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
valset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
train_loader = DataLoader(trainset, batch_size=64, num_workers=2, shuffle=True)
val_loader = DataLoader(valset, batch_size=64, num_workers=2, shuffle=False)

In [ ]:
epochs = 30
lrs = [3e-4, 4e-4, 5e-4]
gammas = [0.8, 0.9]
hdims = [64, 128, 256]
best_accs = []
for lr in lrs:
    for gamma in gammas:
        for hdim in hdims:
            torch.manual_seed(42)
            best_acc = 0
            # Define model
            model = SineKAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            # Define optimizer
            optimizer = optim.AdamW(model.parameters(), lr=lr)
            # Define learning rate scheduler
            scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

            # Define loss
            criterion = nn.CrossEntropyLoss()
            for epoch in range(epochs):
                # Train
                model.train()
                with tqdm(train_loader) as pbar:
                    for i, (images, labels) in enumerate(pbar):
                        images = images.view(-1, 28 * 28).to(device)
                        optimizer.zero_grad()
                        output = model(images)
                        loss = criterion(output, labels.to(device))
                        loss.backward()
                        optimizer.step()
                        accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                        pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

                # Validation
                model.eval()
                val_loss = 0
                val_accuracy = 0
                with torch.no_grad():
                    for images, labels in val_loader:
                        images = images.view(-1, 28 * 28).to(device)
                        output = model(images)
                        val_loss += criterion(output, labels.to(device)).item()
                        val_accuracy += (
                            (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                        )
                val_loss /= len(val_loader)
                val_accuracy /= len(val_loader)
                if val_accuracy > best_acc:
                    best_acc = val_accuracy

                # Update learning rate
                scheduler.step()

                print(
                    f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
                )
            best_accs.append(best_acc)
            print(f"LR: {lr} Gamma: {gamma} Hdim: {hdim} Best Accuracy: {best_acc}")

100%|██████████| 938/938 [00:18<00:00, 49.56it/s, accuracy=0.969, loss=0.0922, lr=0.0003]


Epoch 1, Val Loss: 0.21242379619603513, Val Accuracy: 0.9362062101910829


100%|██████████| 938/938 [00:19<00:00, 49.24it/s, accuracy=0.938, loss=0.238, lr=0.00024]


Epoch 2, Val Loss: 0.17234048355892775, Val Accuracy: 0.9500398089171974


100%|██████████| 938/938 [00:18<00:00, 50.14it/s, accuracy=0.969, loss=0.143, lr=0.000192]


...

Epoch 28, Val Loss: 0.08779371127719944, Val Accuracy: 0.9796974522292994


100%|██████████| 938/938 [00:24<00:00, 38.69it/s, accuracy=1, loss=2.21e-5, lr=2.62e-5]


Epoch 29, Val Loss: 0.08805721780997436, Val Accuracy: 0.9798964968152867


100%|██████████| 938/938 [00:24<00:00, 38.01it/s, accuracy=1, loss=2.01e-5, lr=2.36e-5]


Epoch 30, Val Loss: 0.0882182371973993, Val Accuracy: 0.9799960191082803
LR: 0.0005 Gamma: 0.9 Hdim: 256 Best Accuracy: 0.9807921974522293


In [10]:
epochs = 30
wds = [0.01, 0.1, 0.25, 0.5, 0.75, 1.]
wd_accs = np.empty((6, epochs))
for h, wd in enumerate(wds):
    torch.manual_seed(42)
    best_acc = 0
    # Define model
    model = SineKAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=4e-4, weight_decay=wd)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                val_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )

        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        wd_accs[h, epoch] = val_accuracy

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:24<00:00, 37.56it/s, accuracy=1, loss=0.0692, lr=0.0004]


Epoch 1, Val Loss: 0.16268330219848331, Val Accuracy: 0.9529259554140127


100%|██████████| 938/938 [00:18<00:00, 49.86it/s, accuracy=0.906, loss=0.275, lr=0.00036]


Epoch 2, Val Loss: 0.14255919809020154, Val Accuracy: 0.9580015923566879


100%|██████████| 938/938 [00:19<00:00, 48.88it/s, accuracy=1, loss=0.0132, lr=0.000324]


...

Epoch 28, Val Loss: 0.06349251327285792, Val Accuracy: 0.9811902866242038


100%|██████████| 938/938 [00:20<00:00, 45.45it/s, accuracy=0.938, loss=0.0687, lr=2.09e-5]


Epoch 29, Val Loss: 0.06298046139629475, Val Accuracy: 0.9818869426751592


100%|██████████| 938/938 [00:22<00:00, 42.26it/s, accuracy=1, loss=0.0575, lr=1.88e-5]


Epoch 30, Val Loss: 0.06067556608097545, Val Accuracy: 0.9826831210191083


In [11]:
epochs = 30
hdims = [16, 32, 64, 128, 256, 512]
hdim_accs = np.empty((6, epochs))
for h, hdim in enumerate(hdims):
    torch.manual_seed(42)
    # Define model
    model = SineKAN(layers_hidden=[28 * 28, hdim, 10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=4e-4, weight_decay=.5)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                val_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )
        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        hdim_accs[h, epoch] = val_accuracy

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:18<00:00, 49.65it/s, accuracy=0.906, loss=0.269, lr=0.0004]


Epoch 1, Val Loss: 0.31872255487996304, Val Accuracy: 0.9132165605095541


100%|██████████| 938/938 [00:19<00:00, 46.99it/s, accuracy=0.969, loss=0.153, lr=0.00036]


Epoch 2, Val Loss: 0.28569505121677546, Val Accuracy: 0.9208797770700637


100%|██████████| 938/938 [00:18<00:00, 49.96it/s, accuracy=0.875, loss=0.244, lr=0.000324]


...

Epoch 28, Val Loss: 0.044854627632714694, Val Accuracy: 0.9858678343949044


100%|██████████| 938/938 [00:21<00:00, 44.59it/s, accuracy=1, loss=0.00576, lr=2.09e-5]


Epoch 29, Val Loss: 0.04531116210829591, Val Accuracy: 0.9853702229299363


100%|██████████| 938/938 [00:21<00:00, 42.77it/s, accuracy=1, loss=0.00278, lr=1.88e-5]


Epoch 30, Val Loss: 0.04356003303425038, Val Accuracy: 0.9859673566878981


In [12]:
epochs = 30
n_layers = [1, 2, 3, 4]
layer_accs = np.empty((4, epochs))
for h, n_layer in enumerate(n_layers):
    torch.manual_seed(42)
    # Define model
    model = SineKAN(layers_hidden=[28 * 28] + [128]*n_layer + [10], grid_size=8)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=4e-4, weight_decay=.5)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                val_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )
        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        layer_accs[h, epoch] = val_accuracy

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:18<00:00, 50.97it/s, accuracy=0.906, loss=0.253, lr=0.0004]


Epoch 1, Val Loss: 0.2376669420724272, Val Accuracy: 0.9300358280254777


100%|██████████| 938/938 [00:19<00:00, 47.29it/s, accuracy=0.969, loss=0.0905, lr=0.00036]


Epoch 2, Val Loss: 0.140094636281015, Val Accuracy: 0.9607882165605095


100%|██████████| 938/938 [00:18<00:00, 50.45it/s, accuracy=0.938, loss=0.227, lr=0.000324]


...

Epoch 28, Val Loss: 0.08093475776471805, Val Accuracy: 0.9778065286624203


100%|██████████| 938/938 [00:21<00:00, 42.98it/s, accuracy=0.969, loss=0.0469, lr=2.09e-5]


Epoch 29, Val Loss: 0.07385839822394712, Val Accuracy: 0.9802945859872612


100%|██████████| 938/938 [00:21<00:00, 43.97it/s, accuracy=1, loss=0.00157, lr=1.88e-5]


Epoch 30, Val Loss: 0.07221318118348455, Val Accuracy: 0.9795979299363057


In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def profile_model(model, inputs, num_runs=100):
    total_time = 0
    for _ in range(num_runs):
        with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
            model(inputs)

        # Extract total CUDA time spent on 'model_inference'
        for event in prof.key_averages():
            total_time += event.cuda_time_total

    average_time = total_time / num_runs
    print(f"Average inference time: {average_time} microseconds")

In [ ]:
inputs = torch.randn((1, 784)).to(device)
hdims = [16, 32, 64, 128, 256, 512]
for h in hdims:
    model = SineKAN(layers_hidden=[28 * 28, h, 10], grid_size=8).to(device)
    print('Hidden Dims: %d' % h)
    profile_model(model, inputs, 1000)

In [ ]:
hdims = [16, 32, 64, 128, 256, 512]
for h in hdims:
    model = SineKAN(layers_hidden=[28 * 28, 64, 10], grid_size=8).to(device)
    inputs = torch.randn((h, 784)).to(device)
    print('Input Batch Size: %d' % h)
    profile_model(model, inputs, 1000)

In [ ]:
nlayers = [1, 2, 3, 4]
for n in nlayers:
    model = SineKAN(layers_hidden=[28 * 28] + [64]*n +[10], grid_size=8).to(device)
    inputs = torch.randn((1, 784)).to(device)
    print('Number of Hidden Layers: %d' % n)
    profile_model(model, inputs, 1000)